## Data Science - Final Project

I am using a Kaggle dataset [Reviews of Universal Studios](https://www.kaggle.com/dwiknrd/reviewuniversalstudio).

I will analyze the reviews and plot the ratings over time from 2012 - 2020.

Then analyze the review text to determine the sentiment of the review and compare to the rating given.

### Imports

In [1]:
# Install dependencies required to export Plotly plots as images
# ! pip install -U kaleido

In [2]:
import calendar
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, \
precision_score, recall_score

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

from IPython.display import display

Set options to ignore all *Future Warning* messages.

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

We will use the VADER (Valence Aware Dictionary for Sentiment Reasoning) model from NLTK.

In [4]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\fmcguirk\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Read data from source file.

In [5]:
df_reviews = pd.read_csv('../data/universal_studio_branches.csv')

## Exploratory data analysis

Check the data in the dataset.

In [6]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50904 entries, 0 to 50903
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   reviewer      50904 non-null  object 
 1   rating        50904 non-null  float64
 2   written_date  50904 non-null  object 
 3   title         50904 non-null  object 
 4   review_text   50904 non-null  object 
 5   branch        50904 non-null  object 
dtypes: float64(1), object(5)
memory usage: 2.3+ MB


In [7]:
df_reviews.isna().sum()

reviewer        0
rating          0
written_date    0
title           0
review_text     0
branch          0
dtype: int64

In [8]:
df_reviews['rating'].value_counts()

5.0    28202
4.0    13514
3.0     5229
2.0     1986
1.0     1973
Name: rating, dtype: int64

Change the rating column to integer since all values are integer.

In [9]:
df_reviews['rating'] = df_reviews['rating'].apply(lambda x: int(x))

Perform data cleanup
* create a new date column by converting the values in the written date column
* Remove 'Universal Studios' from branch names
* Drop columns we will not be using

In [10]:
df_reviews['date'] = pd.to_datetime(df_reviews['written_date'])

In [11]:
df_reviews['branch'] = [x.replace('Universal Studios ', '') for x in df_reviews['branch']]

In [12]:
# drop columns we will not use
df_reviews = df_reviews.drop(['reviewer', 'written_date'], axis=1)

In [13]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50904 entries, 0 to 50903
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   rating       50904 non-null  int64         
 1   title        50904 non-null  object        
 2   review_text  50904 non-null  object        
 3   branch       50904 non-null  object        
 4   date         50904 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 1.9+ MB


Create a new dataframe with only the reviews for Universal Studios Florida for July 2019.

In [14]:
park = 'Florida'
year = 2019
month = 7

df_florida_2019_07 = df_reviews[(df_reviews['branch']==park) & \
                      (df_reviews['date'].dt.year == year) & (df_reviews['date'].dt.month == month)].copy()

In [15]:
print(df_florida_2019_07.shape)
df_florida_2019_07.sort_values(by='rating', ascending=False).head()

(233, 5)


,rating,title,review_text,branch,date
1930,5,Worth Staying at the Hotels,We stayed at the Lowes Sapphire Falls and we h...,Florida,2019-07-14
1952,5,An amazing day,"Important tip, before visiting the Park, downl...",Florida,2019-07-10
1944,5,VIP Tour w/ PJ,We had a 9:30 a.m. public VIP tour with PJ and...,Florida,2019-07-12
1945,5,Familyvacation,Excellent! As a movie fan this is the best par...,Florida,2019-07-12
1946,5,Birthday,I am a big kid at heart! My husband made this ...,Florida,2019-07-11


In [16]:
df_florida_2019_07['rating'].value_counts()

5    122
4     44
3     31
1     22
2     14
Name: rating, dtype: int64

In [17]:
years = df_reviews['date'].dt.year.unique()
years = np.sort(years).tolist()
years

[2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021]

In [18]:
years = np.arange(2012, 2020)
years

array([2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019])

## Analyze Sentiment of Reviews
---
Instantiate the sentiment analyzer, track the overall sentiment of the review text, accumulate the results, and add to the dataset.

In [19]:
df_analyzed_reviews = df_reviews[['rating', 'date', 'title', 'review_text', 'branch']].copy()

In [20]:
neg = []
neu = []
pos = []
sia = SentimentIntensityAnalyzer()
for review in df_analyzed_reviews['review_text']:
    sent = sia.polarity_scores(review)
    pos.append(1 if sent['compound'] >= 0.5 else 0)
    neg.append(1 if sent['compound'] <= -0.5 else 0)
    neu.append(1 if sent['compound'] > -0.5 and sent['compound'] < 0.5 else 0)

In [21]:
df_analyzed_reviews['neg'] = neg
df_analyzed_reviews['neu'] = neu
df_analyzed_reviews['pos'] = pos

In [22]:
print(df_analyzed_reviews.shape)
df_analyzed_reviews.head()

(50904, 8)


,rating,date,title,review_text,branch,neg,neu,pos
0,2,2021-05-30,Universal is a complete Disaster - stick with ...,We went to Universal over Memorial Day weekend...,Florida,1,0,0
1,1,2021-05-30,Food is hard to get.,The food service is horrible. I’m not reviewin...,Florida,1,0,0
2,2,2021-05-30,Disappointed,I booked this vacation mainly to ride Hagrid m...,Florida,0,0,1
3,4,2021-05-29,My opinion,When a person tries the test seat for the ride...,Florida,0,1,0
4,5,2021-05-28,The Bourne Stuntacular...MUST SEE,"Ok, I can't stress enough to anyone and everyo...",Florida,0,0,1


## Export processed data
---

Export the entire dataframe including the review_text column.

In [23]:
df_analyzed_reviews.to_csv('../data/df_analyzed_reviews.csv', index=False)

Export the entire dataframe excluding the review_text column.

In [24]:
df_analyzed_reviews.drop(['review_text'], axis=1).to_csv('../data/df_analyzed_reviews2.csv', index=False)

## Visualization
---

In [25]:
park = 'Florida'
year = 2019
month = 7

df_florida_2019_07 = df_analyzed_reviews[(df_analyzed_reviews['branch']==park) & \
                          (df_analyzed_reviews['date'].dt.year == year) & \
                              (df_analyzed_reviews['date'].dt.month == month)].copy()

In [26]:
df_florida_2019_07.head()

,rating,date,title,review_text,branch,neg,neu,pos
1814,4,2019-07-31,Good fun,We enjoyed our day at universal studios. The p...,Florida,0,0,1
1815,5,2019-07-31,Epic,Absolutely brilliant for all the family! We ca...,Florida,0,0,1
1816,3,2019-07-31,Good but not special,Sorry to say but I have been here before and i...,Florida,1,0,0
1817,4,2019-07-31,Fun Times Despite Teenagers,We visited Universal Studios(US) with two teen...,Florida,0,0,1
1818,4,2019-07-31,Good Time,Overall Universal excellent... I took my 8 yea...,Florida,0,0,1


In [27]:
index_list = [5, 4, 3, 2, 1]
summary_df = pd.DataFrame(columns=['neutral', 'negative', 'positive'], \
                             index=index_list)
for rating in index_list:
    num_positive = df_florida_2019_07[(df_florida_2019_07['rating'] == rating) \
                                    & (df_florida_2019_07['pos'] == 1)]['title'].count()
    num_negative = df_florida_2019_07[(df_florida_2019_07['rating'] == rating) \
                                    & (df_florida_2019_07['neg'] == 1)]['title'].count()
    num_neutral = df_florida_2019_07[(df_florida_2019_07['rating'] == rating) \
                                    & (df_florida_2019_07['neu'] == 1)]['title'].count()
    
    summary_df.loc[rating] = [num_neutral, num_negative, num_positive]
summary_df.head()

,neutral,negative,positive
5,9,1,112
4,5,1,38
3,12,2,17
2,2,4,8
1,6,9,7


In [28]:
data = []
for col in summary_df.columns.to_list():
    data.append(
        go.Scatter(
            x=summary_df.index,
            y=summary_df[col],
            name=col
        )
    )

data

[Scatter({
     'name': 'neutral', 'x': array([5, 4, 3, 2, 1], dtype=int64), 'y': array([9, 5, 12, 2, 6], dtype=object)
 }),
 Scatter({
     'name': 'negative', 'x': array([5, 4, 3, 2, 1], dtype=int64), 'y': array([1, 1, 2, 4, 9], dtype=object)
 }),
 Scatter({
     'name': 'positive', 'x': array([5, 4, 3, 2, 1], dtype=int64), 'y': array([112, 38, 17, 8, 7], dtype=object)
 })]

In [29]:
fig=go.Figure(data=data)
fig.show()

In [30]:
def check_sentiment(park, year, month):
    # 
    df_park_month = df_analyzed_reviews[(df_analyzed_reviews['branch']==park) & \
                          (df_analyzed_reviews['date'].dt.year == year) & \
                              (df_analyzed_reviews['date'].dt.month == month)].copy()
    
    print('Reviews for Universal Studio ', park, ' for ', year, '/', month)
    print('  Total reviews analyzed: ', df_park_month.shape[0])


    summary_cols = ['neutral', 'negative', 'positive']
    summary_index = [5, 4, 3, 2, 1]

    summary_df = pd.DataFrame(columns=summary_cols, \
                             index=summary_index)

    for rating in summary_index:
#        num_reviews = df_park_month[(df_park_month['rating'] == rating)]['title'].count()
        num_positive = df_park_month[(df_park_month['rating'] == rating) \
                                     & (df_park_month['pos'] == 1)]['title'].count()
        num_negative = df_park_month[(df_park_month['rating'] == rating) \
                                     & (df_park_month['neg'] == 1)]['title'].count()
        num_neutral = df_park_month[(df_park_month['rating'] == rating) \
                                     & (df_park_month['neu'] == 1)]['title'].count()
        
#        summary_df.loc[rating] = [num_reviews, num_positive, num_negative, num_neutral]
        summary_df.loc[rating] = [num_neutral, num_negative, num_positive]

    display("Summary DataFrame", summary_df)

    data = []
    for col in summary_df.columns.to_list():
        data.append(
            go.Scatter(
                x=summary_df.index,
                y=summary_df[col],
                name=col
            )
        )

    fig = go.Figure(data)

    fig.update_layout(
        xaxis=dict(tickformat='0.0', title='Rating'),
        yaxis=dict(title='Total Reviews'),
        title=dict(text='Universal Studios ' + park + ' Review Sentiment ' + calendar.month_name[month] + ' ' + str(year)),
        legend=dict(title='Sentiment')
    )

    return fig

In [31]:
fig = check_sentiment('Florida', 2019, 7)
#fig.write_html('sentiment-florida-2019-07.html')
#fig.write_image('sentiment-florida-2019-07.png')
fig.show()

Reviews for Universal Studio  Florida  for  2019 / 7
  Total reviews analyzed:  233


'Summary DataFrame'

,neutral,negative,positive
5,9,1,112
4,5,1,38
3,12,2,17
2,2,4,8
1,6,9,7


## Visualize using Saved Data
---
The processed dataframe was saved to a CSV file both with the review_text column and without.

We will load the CVS file without the review_text columns, process the data to set values to expected data types and then attempt to visualize again.

In [32]:
df_analyzed_reviews = pd.read_csv('../data/df_analyzed_reviews2.csv')

In [33]:
df_analyzed_reviews['date'] = pd.to_datetime(df_analyzed_reviews['date'])

In [34]:
df_analyzed_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50904 entries, 0 to 50903
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   rating  50904 non-null  int64         
 1   date    50904 non-null  datetime64[ns]
 2   title   50904 non-null  object        
 3   branch  50904 non-null  object        
 4   neg     50904 non-null  int64         
 5   neu     50904 non-null  int64         
 6   pos     50904 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 2.7+ MB


Now, try to visualize using the same parameters as before.

In [35]:
fig = check_sentiment('Florida', 2019, 7)
#fig.write_html('sentiment-florida-2019-07.html')
#fig.write_image('sentiment-florida-2019-07.png')
fig.show()

Reviews for Universal Studio  Florida  for  2019 / 7
  Total reviews analyzed:  233


'Summary DataFrame'

,neutral,negative,positive
5,9,1,112
4,5,1,38
3,12,2,17
2,2,4,8
1,6,9,7


In [36]:
summary_df

,neutral,negative,positive
5,9,1,112
4,5,1,38
3,12,2,17
2,2,4,8
1,6,9,7


In [39]:
summary_df.columns.to_list()

['neutral', 'negative', 'positive']

In [44]:
summary_df.iloc[-1,1]

9

In [46]:
[summary_df.index[0], summary_df.columns.to_list()]

[5, ['neutral', 'negative', 'positive']]

In [48]:
list=[]
list.append(summary_df.index)
list

[Int64Index([5, 4, 3, 2, 1], dtype='int64')]